# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the **establishments.json** file from the Terminal:
 - switch to proper directory, where the respective file is saved: <br>
   ```cd <path to the 'nosql-challenge' folder>/Resources```
 - check if the establishments.json file is there: <br>
   e.g. ```ls``` if using Mac
 - import the data to a `uk_food` database and `establishments` collection, dropping any `establishments` collection if such already exists: <br>
    ```mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json```

In [1]:
# Create the function for checking if all the data (which are not "None") under key <el_key> have type <el_type> within the collection variable <collection_v>
def check_if_converted(collection_v,el_key,el_type):
    type_e = collection_v.count_documents({"$and":[{el_key:{"$type": el_type}},
                                                   {el_key:{"$exists": True}},
                                                   {el_key:{"$ne":None}}]})
    existing_e = collection_v.count_documents({"$and":[{el_key: {"$exists": True}},
                                                       {el_key:{"$ne":None}}]})
    if type_e==existing_e: print(f"All '{el_key}' are converted to a '{el_type}' type.")
    else: 
        print(f"There are documents with '{el_key}' not converted to a '{el_type}' type.")
    print(f" -- details: there {type_e} '{el_key}' elements with proper type out of {existing_e} existing '{el_key}' elements, which are not None. \n")

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'local', 'met', 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [7]:
# assign the collection to a variable
establishments=db["establishments"]

In [8]:
# review a document in the establishments collection
pprint(establishments.find_one({}))

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('655912fb7249374d19037347'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_document={"BusinessName":"Penang Flavours",
              "BusinessType":"Restaurant/Cafe/Canteen",
              "BusinessTypeID":"",
              "AddressLine1":"Penang Flavours",
              "AddressLine2":"146A Plumstead Rd",
              "AddressLine3":"London",
              "AddressLine4":"",
              "PostCode":"SE18 7DY",
              "Phone":"",
              "LocalAuthorityCode":"511",
              "LocalAuthorityName":"Greenwich",
              "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
              "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
              "scores":{
                  "Hygiene":"",
                  "Structural":"",
                  "ConfidenceInManagement":""
              },
              "SchemeType":"FHRS",
              "geocode":{
                  "longitude":"0.08384000", 
                  "latitude":"51.49014200"
              },
              "RightToReply":"",
              "Distance":4623.9723280747176,
              "NewRatingPending":True
             }

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_document)

InsertOneResult(ObjectId('6559130b8a265ab797fdc398'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6559130b8a265ab797fdc398'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query={"BusinessType":"Restaurant/Cafe/Canteen"}
fields={"BusinessTypeID":1, "BusinessType":1, "_id":0}
pprint(establishments.find_one(query, fields))

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
rcc_business_type_id=establishments.find_one(query, fields)['BusinessTypeID']
establishments.update_one({"BusinessName":"Penang Flavours",
                           "geocode":{"longitude":"0.08384000","latitude":"51.49014200"}}, #added geocode, as it identifies new restaurant more uniquely
                          {"$set":{"BusinessTypeID":rcc_business_type_id}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
penang_flavours_count=establishments.count_documents({"BusinessName":"Penang Flavours"})
if penang_flavours_count==1:
    penang_flavours=establishments.find_one({"BusinessName":"Penang Flavours"})
    penang_flavours_type=penang_flavours['BusinessTypeID']
    penang_flavours_name=penang_flavours['BusinessName']
    if penang_flavours_type==1:
        print(f"Update successful: The Business '{penang_flavours_name}' has BusinessTypeID value equal to {penang_flavours_type}")
    else: print("Issue: still wrong Business Type for {penang_flavours_name}")
elif penang_flavours_count>1: print("seems that extra record was added, instead of update")
else: print("record disappeared")

Update successful: The Business 'Penang Flavours' has BusinessTypeID value equal to 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_like_documents_count=establishments.count_documents({"LocalAuthorityName":{"$regex":"Dover"}}) #looking for all the documents with LocalAuthorityName consisting from %Dover%, even though it seems that all of them have actually "Dover" one, but not the variations
dover_like_documents_count

994

In [16]:
# Find how many documents we have in our collection
initial_documents_count=establishments.count_documents({})
initial_documents_count

39780

In [17]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName":{"$regex":"Dover"}})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [18]:
#Check if proper amount of documents left, having in mind the count of documents with "Dover" in LocalAuthorityName
diff=initial_documents_count-dover_like_documents_count

if establishments.count_documents({})==diff: print(f"Proper amount of documents was deleted. Now we have {diff} documents.")
else: print("Wrong amount of documents deleted.")    

Proper amount of documents was deleted. Now we have 38786 documents.


In [19]:
# Check if any remaining documents include Dover
if establishments.find_one({"LocalAuthorityName":{"$regex":"Dover"}})==None:
    print("No documents with LocalAuthorityName like 'Dover' anymore.")
else: print("There are still documents with LocalAuthorityName like 'Dover'.")

No documents with LocalAuthorityName like 'Dover' anymore.


In [20]:
# Check that other documents remain with 'find_one'
if establishments.find_one()!= None: print("At least one document still exists in the collection!")
else: print("Collection is empty!")    


At least one document still exists in the collection!


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [21]:
# Change the data type from String to Decimal for longitude and latitude
# Check if the above data is convertable and if so convert
temp=[i["geocode"] for i in establishments.find({},{"geocode.latitude":1,"geocode.longitude":1, "_id":0})]
is_convertable=True
try: 
    pd.DataFrame(temp).astype("float64")
except: 
    is_convertable=False
    print("There are issues with data. Please, investigate and adjust the data before convert.")

if is_convertable:
    establishments.update_many({},
    [{
        "$set": {"geocode.latitude": {"$convert": { "input": "$geocode.latitude", "to": "decimal"}},
                "geocode.longitude": {"$convert": { "input": "$geocode.longitude", "to": "decimal"}}}
    }])
    print("Data types were updated successfully.")

Data types were updated successfully.


Use `update_many` to convert `RatingValue` to integer numbers.

In [22]:
# Find all the non 1-5 Rating Values
non_ratings=[]
for i in establishments.find({"RatingValue":{"$exists":True}},{"RatingValue":1, "_id":0}):
    if i["RatingValue"] not in ["1","2","3","4","5"]: non_ratings.append(i["RatingValue"])
non_ratings=list(pd.unique(non_ratings))
non_ratings

['AwaitingInspection',
 'Exempt',
 '0',
 'AwaitingPublication',
 'Awaiting Inspection',
 'Pass']

In [23]:
# Set non 1-5 Rating Values to Null
establishments.update_many({"RatingValue": {"$in": non_ratings}}, 
                           [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4156, 'nModified': 4156, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [24]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},
    [{
        "$set": {"RatingValue": {"$convert": { "input": "$RatingValue", "to": "int"}}}
    }])

UpdateResult({'n': 38786, 'nModified': 34630, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [25]:
# Check that the coordinates and rating value are now numbers
check_if_converted(establishments,"geocode.latitude","decimal")
check_if_converted(establishments,"geocode.longitude","decimal")
check_if_converted(establishments,"RatingValue","int")



All 'geocode.latitude' are converted to a 'decimal' type.
 -- details: there 38786 'geocode.latitude' elements with proper type out of 38786 existing 'geocode.latitude' elements, which are not None. 

All 'geocode.longitude' are converted to a 'decimal' type.
 -- details: there 38786 'geocode.longitude' elements with proper type out of 38786 existing 'geocode.longitude' elements, which are not None. 

All 'RatingValue' are converted to a 'int' type.
 -- details: there 34629 'RatingValue' elements with proper type out of 34629 existing 'RatingValue' elements, which are not None. 



All data are converted properly. Note, that there are 34629 values within 'RatingValue', which are not None, and there are 34630 updated, when changing data type to integer. The one which makes the difference is new record for "BusinessName":"Penang Flavours", that was entered without rating value and updated by ading the field, during data type update, with None value.